In [1]:
using CSV
using DataFrames
using JLD2
using LinearAlgebra
using ProgressMeter
using Random
using SparseArrays
using StatsBase

In [2]:
Random.seed!(1234)

TaskLocalRNG()

In [3]:
df_neurons = CSV.read("./data/ol_columns.csv", DataFrame; header=true);

In [4]:
N = nrow(df_neurons)
K = length(unique(df_neurons[!, "column id"])) - 1
T = length(unique(df_neurons[!, "cell type"]))
N, K, T

(23452, 796, 31)

In [5]:
d_neuron = Dict(df_neurons[!, "cell id"] .=> 1:N)
d_type = Dict(unique(df_neurons[!, "cell type"]) .=> 1:T);

In [6]:
@time begin
    X = zeros(Int8, N, K)
    Y = zeros(Int8, N, T)

    for idx in 1:N
        row = df_neurons[idx, :]
        Y[idx, d_type[row["cell type"]]] = 1
        if row["column id"] != "not assigned"
            col = parse(Int, row["column id"])
            X[idx, col] = 1
        end
    end
end

  0.126217 seconds (436.58 k allocations: 32.506 MiB, 82.39% compilation time)


In [7]:
df_conn = CSV.read("./data/ol_connections.csv", DataFrame);

In [8]:
@time begin
    W = zeros(Int16, N, N)

    for row in eachrow(df_conn)
        i = row["from cell id"]
        j = row["to cell id"]
        w = row["synapses"]
        W[d_neuron[i], d_neuron[j]] = w
    end
end

  0.288091 seconds (3.80 M allocations: 1.106 GiB, 4.34% gc time, 5.13% compilation time)


In [9]:
function f(X, W)
    X_sparse = sparse(X)
    W_sparse = sparse(W)
    return sum(W_sparse .* (X_sparse * X_sparse'))
end

f (generic function with 1 method)

In [10]:
orig_f = f(X, W)
curr_f = orig_f

1381915

In [11]:
types_of_interest = ["R7", "R8", "T2a", "T3", "Tm3"]
type_list = [d_type[cell_type] for cell_type in types_of_interest]

5-element Vector{Int64}:
 29
 31
 14
 13
 12

In [12]:
function find_column(X, idx)
    # Find the column of a given neuron
    col_list = findall(X[idx, :] .!= 0)
    if length(col_list) == 1
        return col_list[1]
    elseif isempty(col_list)
        return NaN
    else
        error("Something wrong with column assignment")
    end
end

function find_colmates(X, k)
    # Find all the neurons in a given column k ("column mates")
    if isnan(k)
        return Int8[]
    else
        colmates = findall(X[:, k] .== 1)
        return colmates
    end
end

function update_colmates(colmates, idx_remove, idx_add)
    # Update the colmates after swap
    if isempty(colmates)
        return Int8[]
    end
    s_1 = Set(colmates)
    s_2 = Set([idx_remove])
    s_3 = Set([idx_add])
    s = setdiff(s_1, s_2) ∪ s_3
    return collect(s)
end

function delta_f(X, W, idx_1, idx_2)
    # Calculate the difference in objective function after the swap
    col_1 = find_column(X, idx_1)
    col_2 = find_column(X, idx_2)
    
    # Pre swap
    colmates_1_pre = find_colmates(X, col_1)
    colmates_2_pre = find_colmates(X, col_2)
    loss_1 = sum(W[idx_1, colmates_1_pre]) + sum(W[colmates_1_pre, idx_1]) - W[idx_1, idx_1]
    loss_2 = sum(W[idx_2, colmates_2_pre]) + sum(W[colmates_2_pre, idx_2]) - W[idx_2, idx_2]
    
    # Post swap
    colmates_1_post = update_colmates(colmates_2_pre, idx_2, idx_1)
    colmates_2_post = update_colmates(colmates_1_pre, idx_1, idx_2)
    gain_1 = sum(W[idx_2, colmates_2_post]) + sum(W[colmates_2_post, idx_2]) - W[idx_2, idx_2]
    gain_2 = sum(W[idx_1, colmates_1_post]) + sum(W[colmates_1_post, idx_1]) - W[idx_1, idx_1]
    
    d_f = (gain_1 + gain_2) - (loss_1 + loss_2)
    return d_f
end

delta_f (generic function with 1 method)

In [13]:
function swap_rows!(X, idx_1, idx_2)
    X[idx_1, :], X[idx_2, :] = X[idx_2, :], X[idx_1, :]
end

swap_rows! (generic function with 1 method)

In [14]:
@time begin
    swap_list = []

    @showprogress for trial in 1:100000000
        # t = sample(1:T, 1)
        t = sample(type_list, 1)
        candidate_neurons = findall(dropdims(Y[:, t] .== 1, dims=2))
        idx_1, idx_2 = sample(candidate_neurons, 2, replace=false)

        d_f = delta_f(X, W, idx_1, idx_2)
        if d_f > 0
            swap_rows!(X, idx_1, idx_2)
            curr_f += d_f
            d_f = curr_f - orig_f
            println("Trial: $trial, Curr_f: $curr_f, d_f: $d_f")
            push!(swap_list, (idx_1, idx_2))

            file_path = joinpath("../results", "swap_list_$(Int(curr_f)).jld2")
            @save file_path swap_list
        end
    end
end


Progress:   0%|                                         |  ETA: 2:51:53

Trial: 48347, Curr_f: 1381927, d_f: 12


Progress:   0%|                                         |  ETA: 3:02:21

Trial: 68378, Curr_f: 1381943, d_f: 28


Progress:   0%|                                         |  ETA: 2:32:57

...

Trial: 10507612, Curr_f: 1382833, d_f: 918


Progress:  11%|████▍                                    |  ETA: 1:43:35

Trial: 10738103, Curr_f: 1382889, d_f: 974


Progress: 100%|█████████████████████████████████████████| Time: 3:40:25


13225.725133 seconds (10.79 G allocations: 10.096 TiB, 5.49% gc time, 0.02% compilation time: 35% of which was recompilation)


In [15]:
f(X, W)

1382889